## Adicionando o path para utilizar a bibliotecas necessárias

In [1]:
from sys import path
dir_path: str = %pwd
dir_path = dir_path.replace("/Simulations", "")

path.append(dir_path)

## Fazendo import das dependências

In [2]:
from BHA_functions import asyncSimulations_Linux
from BHA_functions import DataCollector, DataGroup
from BHA_functions import GraphicGenerator
from datetime import datetime

## Definindo as constantes das simulações

In [3]:
PATH: str = f'{dir_path}/Simulations_Data/topology_simulations'
GRAPHICS_PATH: str = f"{dir_path}/Graphics/topology_simulations/"

CORES: int = 12
RUNS_PER_POINT: int = 100

TARGET: tuple[bool, bool] = (True, False)

NETWORK_PROB: float = 0.8
BHA_PROB: float = 0.4
BHA_PROP: float = 0.2

TOPOLOGIES: tuple[str, str, str] = ('Grade', 'Ba', 'Er')
TOPOLOGY_PARAMS: tuple[float, float, float] = (0.1, 0.3, 0.5)

PASS_OF_NODE: int = 12
NUMBER_OF_NODES: tuple[int,...] = tuple([i for i in range(12, 108, PASS_OF_NODE)])
GRADE_NODES: dict = {
    12:(3, 4),
    24:(4, 6),
    36:(6, 6),
    48:(6, 8),
    60:(6, 10),
    72:(8, 9),
    84:(7, 12),
    96:(8, 12)
}

GRAPHIC_POINTS: int = len(NUMBER_OF_NODES)

## Criando os Parâmetros das simulações

In [4]:
# Default params
simulations_params: dict = {
            'runs':RUNS_PER_POINT,
            'topology':TOPOLOGIES[0],
            'number_nodes':20,
            'topology_args':(4,3),
            'entanglements_replanished':10,
            'requests':100,
            'attempts_per_request':2,
            'network_prob':NETWORK_PROB,
            'num_black_holes':0,
            'black_hole_prob':BHA_PROB,
            'black_hole_target':TARGET[0]
        }

## Rodando a simulação sem Black Hole em todas as topologias

In [ ]:
for topology in TOPOLOGIES:
    simulations_params['topology'] = topology

    for indexgroup, topology_param in enumerate(TOPOLOGY_PARAMS):
        if topology == 'Grade' and indexgroup > 0: # Grade topology has no other parameters
            continue

        for point, number_of_nodes in enumerate(NUMBER_OF_NODES):
            simulations_params['number_nodes'] = number_of_nodes

            if topology == 'Grade':
                simulations_params['topology_args'] = GRADE_NODES[number_of_nodes]
            elif topology == 'Ba':
                simulations_params['topology_args'] = (int(topology_param*10),)
            elif topology == 'Er':
                simulations_params['topology_args'] = (topology_param,)

            start: datetime = datetime.now()
            print(f"Topologia {topology} no Grupo {indexgroup} no ponto: {point}")
            # Runing no BHA simulation
            default_simulation: DataCollector = asyncSimulations_Linux(cores=CORES, **simulations_params)
            if topology == "Grade":
                default_simulation.save(file_name=f'{PATH}/Default_Network/{topology}/point{point}')
            else:
                default_simulation.save(file_name=f'{PATH}/Default_Network/{topology}/{int(topology_param*10)}param/point{point}')
            print(f"Terminou no tempo de: {datetime.now()-start}")
            print('-='*50)


## Realizando todas as simulações com Black Holes

In [ ]:
counter = 0
# Run all topologies and update your parameter
for topology in TOPOLOGIES:
    simulations_params['topology'] = topology
    
    # Run all targets (True, False) and update your parameter
    for target in TARGET:
        simulations_params['black_hole_target'] = target
        
        # Run all topology parameters (0.1, 0.3, 0.5) and updating your parameters
        for indexgroup, topology_param in enumerate(TOPOLOGY_PARAMS):
            
            # Grade topology won't change your params, so no need run more than once simulation per target
            if topology == "Grade" and indexgroup > 0:
                continue

            # Updating parameters of topology (0.1, 0. 3 or 0.5) in prob_edge_creation and (1, 3 or 5) in edges_to_attach
            if topology == 'Er':
                simulations_params['topology_args'] = (topology_param,)

            elif topology == 'Ba':
                simulations_params['topology_args'] = (int(topology_param * 10),)

            # Run all points and update your parameter
            for point, number_of_nodes in enumerate(NUMBER_OF_NODES):
                start: datetime = datetime.now() # Debug: start timer to BenchMark
                
                simulations_params['number_nodes'] = number_of_nodes

                # Update the topology parameters, because number_nodes don't affect this topology
                if topology == 'Grade':
                    simulations_params['topology_args'] = GRADE_NODES[number_of_nodes]

                    
                # Updating umber of nodes of simulation
                simulations_params['num_black_holes'] = int(number_of_nodes * BHA_PROP)

                # Running simulation
                simulations_dc: DataCollector = asyncSimulations_Linux(cores=CORES, **simulations_params)

                # Saving CSV file
                if topology != "Grade":
                    simulations_dc.save(file_name=f'{PATH}/BHA_Network/{topology}/Target-{target}/{int(topology_param*10)}param/point{point}')
                else:
                    simulations_dc.save(file_name=f'{PATH}/BHA_Network/{topology}/Target-{target}/point{point}')

                counter += 1
                # BenchMark
                print(f"As {simulations_params['runs']} simulações da topologia {topology} finalizaram no tempo de: {datetime.now()-start}")
                print(f"Já rodaram {counter} pontos de 112")
                print('-='*50)

## Coletando os dados diretamente dos CSVs de cada simulação

In [6]:
default_networks: dict[str, DataGroup] = dict()
for topology in TOPOLOGIES:
    temp_dataGroup: DataGroup = DataGroup()

    for indexgroup, topology_param in enumerate(TOPOLOGY_PARAMS):
        if topology == "Grade" and indexgroup > 0: # Grade topology has no other parameters
            continue

        for point in range(0, GRAPHIC_POINTS):
            default_network: DataCollector = DataCollector()
            if topology != "Grade":
                default_network.get_DataFrame_csv(f'{PATH}/Default_Network/{topology}/{int(topology_param*10)}param/point{point}.csv')
            else:
                default_network.get_DataFrame_csv(f'{PATH}/Default_Network/{topology}/point{point}.csv')

            temp_dataGroup.add_Data(value=default_network, indexgroup=indexgroup)

    default_networks[topology] = temp_dataGroup

# Create DataGroups for all params and Topologys
dg_grade_target_off: DataGroup = DataGroup()
dg_grade_target_on: DataGroup = DataGroup()
dg_ba_target_off: DataGroup = DataGroup()
dg_ba_target_on: DataGroup = DataGroup()
dg_er_target_off: DataGroup = DataGroup()
dg_er_target_on: DataGroup = DataGroup()

# Puting all DataGroups in a dict
dg_dict: dict = {
    'Grade':{True:dg_grade_target_on, False:dg_grade_target_off},
    'Ba':{True:dg_ba_target_on, False:dg_ba_target_off},
    'Er':{True:dg_er_target_on, False:dg_er_target_off}
}

# Collecting data of all topologies, except Grade topology
# The DataGroup and dict will look like this
# dg_dict = {
# 'Grade': {True:[(D1,..., Dnpoints)], False: [(D1,..., Dnpoints)]}, 
# 'Ba': {True:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)], False:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)]},
# 'Er': {True:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)], False:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)]}
# }
# D1 and Dnpoints are DataCollectors with all data of current point
#
# {Topology: { True: [ (     D1, ..., Dn), ..., ()], False:[ (),..., ()]}, ...}
# ↑          ↑         ↑     ↑
# ↑          ↑         ↑     DataCollector per point
# ↑          ↑        Param of simulation (0.1, 0.3, 0.5)
# ↑         Dict of tagerts on (True) or off (False)
# Dict of topology

for topology in TOPOLOGIES:
    if topology == "Grade":
        continue
    for target in TARGET:
        for indexgroup, topology_param in enumerate(TOPOLOGY_PARAMS):
            for point in range(0, GRAPHIC_POINTS):
                temp_dc: DataCollector = DataCollector()
                temp_dc.get_DataFrame_csv(f'{PATH}/BHA_Network/{topology}/Target-{target}/{int(topology_param*10)}param/point{point}.csv')
                
                dg_dict[topology][target].add_Data(value=temp_dc, indexgroup=indexgroup)

# Collecting only data of Grade topology        
for target in TARGET:
    dg_dict['Grade'][target].add_Group(value=tuple())
    for point in range(0, GRAPHIC_POINTS):
        temp_dc: DataCollector = DataCollector()
        temp_dc.get_DataFrame_csv(f'{PATH}/BHA_Network/Grade/Target-{target}/point{point}.csv')

        dg_dict["Grade"][target].add_Data(value=temp_dc, indexgroup=0)


## Utilizando o polimorfismo para alterar a função da classe GraphicGenerator

In [7]:
import matplotlib.pyplot as plt     # Imports to gen graphics
import numpy as np                  # Needed to create array

class GraphicGen(GraphicGenerator):
    def __init__(self):
        super().__init__()

    def add_on_plot(
        self, 
        plot_label: str, 
        color: str,
        x_column: tuple[int, int], 
        y_column_name: str, 
        y_standard_deviation: bool, 
        dc: tuple[DataCollector],
        default_simulations: tuple[DataCollector] | None = None) -> None:
        """
        Will add data on plot selected

        Args:
            plot_label (required): Name of plot
            color: Color of the plot
            x_column (required): Tuple with initial value of x, step
            y_column_name (required): Name of y column on DataCollector
            y_standard_deviation (optional): If True, the standard deviation will be used
            dc (required): Tuple with all DataCollectors
            default_simulations (optional): Pass the network without black holes to compare, if None don't affect the result
        """
        y_points: list = []
        if dc:
            self.dataCollectors: tuple[DataCollector] = dc

        temp_error_bar: list = []

        if default_simulations == None:
            default_simulations_list: list = [0 for i in range(0, len(dc))]
        else:
            default_simulations_list: list = list()
            for temp_datacollector in default_simulations:
                result: int = temp_datacollector.arithmetic_Mean(y_column_name)[y_column_name]
                default_simulations_list.append(result)
                
        for dataCollector, default_arithmetic_mean in zip(self.dataCollectors, default_simulations_list):
            dataCollector.standard_Deviation(y_column_name)
            arithmetic_mean: float = dataCollector.arithmetic_Mean(y_column_name)[y_column_name]
            y_points.append(default_arithmetic_mean - arithmetic_mean)


            temp_error_bar.append(dataCollector.standard_deviations[y_column_name])

        x_points: list = []
        temp_x: int = x_column[0]
        for i in range(0, len(y_points)):
            x_points.append(temp_x)
            temp_x += x_column[1]

        x_points_array: np.ndarray = np.array(x_points)
        y_points_array: np.ndarray = np.array(y_points)

        plt.plot(x_points_array, y_points_array, label=plot_label, color=color, marker='.')

        if y_standard_deviation:
            error_bar: np.ndarray = np.array(temp_error_bar)
            plt.errorbar(x_points, y_points, yerr=error_bar, fmt='.', color=color)

## Criando uma função genérica para plotar os gráficos

In [8]:
def plotGraphic(
        dataGroup: DataGroup,
        default_simulations: dict[str, DataGroup], 
        topology: str,
        pass_of_node: int,
        topology_params: tuple[float],
        y_column_name: str, 
        title: str, 
        x_label: str, 
        y_label: str,
        pdf_file: str,
        grid: bool) -> None:
    """
    Will show the simulations graph

    Args:
        dataGroup (required): DataGroup with all DataCollectors
        default_diff (required): Pass the network without black holes to compare, if None don't affect the result
        topology (required): String with name of the selected topology
        pass_of_node (required): Variation of nodes per point
        topology_params (required): Tuple with all parameter of each plot
        y_column_name (required): Name of column data
        title (required): Title of graph
        x_label (required): X axis label
        y_label (required): Y axis label
        pdf_file (optional): Name of pdf file, if don't want save as pdf, pdf_file = ""
        grid (optional): Add a grid on the plot if is True
    """
    
    graphicGen: GraphicGen = GraphicGen()

    colors: tuple[str, str, str] = ("red", "darkorange", "gold")

    for pos, datacollectors in enumerate(dataGroup):

        if topology == 'Ba':
            plot_label: str = f"m={int(topology_params[pos] * 10)}"
        elif topology == 'Er':
            plot_label: str = f"p={topology_params[pos]}"
        else:
            plot_label: str = 'Grade'


        default_sim: tuple[DataCollector] = default_simulations[topology][pos] # access a Group in the DataGroup


        graphicGen.add_on_plot(
            plot_label=plot_label,
            color=colors[pos],
            x_column=(12, pass_of_node), 
            y_column_name=y_column_name, 
            y_standard_deviation=False,
            dc=datacollectors,
            default_simulations=default_sim)

    graphicGen.show_plot(
        title=title,
        x_label=x_label, 
        y_label=y_label,
        pdf_file=pdf_file,
        grid=grid)

## Gerando o gráfico da Topologia de Grade sem alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Grade'][False],
    default_simulations=default_networks,
    topology='Grade',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_Grade_sem_alvo.pdf",
    grid=True
    )

## Gerando o gráfico da Topologia de Grade com alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Grade'][True],
    default_simulations=default_networks,
    topology='Grade',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_Grade_com_alvo.pdf",
    grid=True
    )

## Gerando o Gráfico da Topologia Barabasi-Albert sem alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Ba'][False],
    default_simulations=default_networks,
    topology='Ba',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_Barabasi-Albert_sem_alvo.pdf",
    grid=True
)

## Gerando o Gráfico da Topologia Barabasi-Albert com alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Ba'][True],
    default_simulations=default_networks,
    topology='Ba',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_Barabasi-Albert_com_alvo.pdf",
    grid=True
)

## Gerando o Gráfico da Topologia Erdos Renyi sem alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Er'][False],
    default_simulations=default_networks,
    topology='Er',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_Erdos-Renyi_sem_alvo.pdf",
    grid=True
)

## Gerando o Gráfico da Topologia Erdos Renyi com alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Er'][True],
    default_simulations=default_networks,
    topology='Er',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso',
    pdf_file=f"{GRAPHICS_PATH}Taxa_Sucesso_Erdos-Renyi_com_alvo.pdf",
    grid=True
)